In [1]:
import os
import platform
import pandas as pd
from multiprocess import Pool, cpu_count
from tqdm.notebook import tqdm

from mail_helpers import ingest_emails


pd.options.mode.copy_on_write = True
# pd.read_csv(fname, engine='pyarrow', dtype_backend='pyarrow')
# pd.Series([1, 2, 3, 4], dtype='int64[pyarrow]')
# pd.Series(['foo', 'bar', 'foobar'], dtype='string[pyarrow]')
# pd.read_csv(my_file, use_nullable_dtypes=True)


Enron Email Dataset

https://www.cs.cmu.edu/~./enron/

In [2]:
# Message files end with a dot. This is not supported on Windows.
# Workaround for Windows:
# Add the prefix "\\?\" to the path to force the OS to read the files.
# normpath() is used everywhere to make sure things remain portable.
cwd = os.getcwd()
if platform.system() == "Windows":
    cwd = "\\\\?\\" + cwd
basedir = cwd + os.path.normpath("/maildir")
top_level_dirs = os.listdir(basedir)

# df = ingest_emails((basedir, "zufferli-j"))

if __name__ == "__main__":
    cpus = cpu_count()
    arglist = list(zip([basedir] * len(top_level_dirs), top_level_dirs))
    with Pool(processes=cpus) as mp_pool:
        mp_results = mp_pool.map_async(ingest_emails, arglist)
        mp_results.wait()
    all_emails = pd.concat([r for r in mp_results.get()], ignore_index=True)
    all_emails["Date"] = pd.to_datetime(all_emails["Date"], unit="s")
    del mp_results
    del mp_pool


TypeError: Tuple or struct_time argument required

In [ ]:
if os.path.exists("all_emails.csv"):
    os.remove("all_emails.csv")
all_emails.to_csv("all_emails.csv", index=False)


In [ ]:
# all_emails_new = pd.read_csv("all_emails.csv", engine='pyarrow')
all_emails_new = pd.read_csv("all_emails.csv")
all_emails_new.head()


,Top_Level_Folder,Mail_folder,Message_File,From,To,Cc,Bcc,Date,Subject,Body,Body_Message,Body_Quoted
0,allen-p,all_documents,1,['1.11913372.-2@multexinvestornetwork.com'],['pallen@enron.com'],[],[],976761660.0,"December 14, 2000 - Bear Stearns' predictions ...",In today's Daily Update you'll find free repor...,In today's Daily Update you'll find free repor...,NaN
1,allen-p,all_documents,10,['messenger@ecm.bloomberg.com'],[],[],[],976725300.0,Bloomberg Power Lines Report,Here is today's copy of Bloomberg Power Lines....,Here is today's copy of Bloomberg Power Lines....,NaN
2,allen-p,all_documents,100,['phillip.allen@enron.com'],['keith.holst@enron.com'],[],[],971100960.0,Consolidated positions: Issues & To Do list,---------------------- Forwarded by Phillip K ...,---------------------- Forwarded by Phillip K ...,NaN
3,allen-p,all_documents,101,['phillip.allen@enron.com'],['keith.holst@enron.com'],[],[],971100000.0,Consolidated positions: Issues & To Do list,---------------------- Forwarded by Phillip K ...,---------------------- Forwarded by Phillip K ...,NaN
4,allen-p,all_documents,102,['phillip.allen@enron.com'],['david.delainey@enron.com'],[],[],970752360.0,NaN,"Dave, \n\n Here are the names of the west desk...","Dave, \n\n Here are the names of the west desk...",NaN


In [ ]:
all_emails_new.dtypes


Top_Level_Folder     object
Mail_folder          object
Message_File          int64
From                 object
To                   object
Cc                   object
Bcc                  object
Date                float64
Subject              object
Body                 object
Body_Message         object
Body_Quoted          object
dtype: object

In [ ]:
all_emails.dtypes


Top_Level_Folder            object
Mail_folder                 object
Message_File                 int64
From                        object
To                          object
Cc                          object
Bcc                         object
Date                datetime64[ns]
Subject                     object
Body                        object
Body_Message                object
Body_Quoted                 object
dtype: object